In [2]:
# Reference: https://github.com/bilguun/Singapore_Dengue

In [ ]:
# Import relevant modules
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import itertools
import numpy

In [3]:
def feature_to_array(file_n):
    im = mpimg.imread(file_n)  
    im_greyscale = rgb2gray(im)   
    imarray = numpy.array(im_greyscale)
    feature=[i for a in imarray for i in a]
    return feature

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def read_pictures(filename):
    file_name=filename
    im = mpimg.imread(file_name)  
    im_greyscale = rgb2gray(im)  

In [4]:
file_name='images/BusStop.png'
bus=feature_to_array(file_name) # array1

file_name='images/Mosquito.png'
mosquito=feature_to_array(file_name) # array4

file_name='images/Str_Density.png'
street=feature_to_array(file_name) # array5

file_name='images/Trash.png'
trash=feature_to_array(file_name) # array6

file_name='images/Case.png'
case=feature_to_array(file_name) # array2

file_name='images/Land.png'
land=feature_to_array(file_name) # array3

In [5]:
#cases = list(case/np.max(case)) # Normalize
inorout = list(1-(land/np.max(land))) # get binary value indicating whether pixel is inside Singapore or not

In [6]:
Data = pd.DataFrame([bus,street,trash,mosquito,case]).T
Data.head()

,0,1,2,3,4
0,0.941176,0.941176,0.941176,0.941176,0.941176
1,0.941176,0.941176,0.941176,0.941176,0.941176
2,0.941176,0.941176,0.941176,0.941176,0.941176
3,0.941176,0.941176,0.941176,0.941176,0.941176
4,0.941176,0.941176,0.941176,0.941176,0.941176


In [7]:
Data_filt = Data[np.array(inorout)==1.0] # filter dataset to only pixels within Singapore
Data_filt.columns = ['BusStop','Street','Trash','Mosquito','Case']
Data_filt['Case'] = np.where(Data_filt['Case']==0,1,0) # Transform Case label into 1 (=case) or 0 (=no case)

/home/harry/SENG3011/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
Data_filt.head()

,BusStop,Street,Trash,Mosquito,Case
147867,0.941176,0.941176,0.941176,0.941176,0
150112,0.941176,0.941176,0.941176,0.941176,0
150113,0.941176,0.941176,0.941176,0.941176,0
150114,0.941176,0.941176,0.941176,0.941176,0
151167,0.219608,0.121569,0.000000,0.941176,0


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import statsmodels.formula.api as smf

ModuleNotFoundError: No module named 'sklearn.grid_search'

In [ ]:
X = Data_filt[['BusStop','Street','Trash','Mosquito']]
Y = Data_filt['Case']

X_train, X_test = train_test_split(X, test_size=0.2, random_state=21)
Y_train, Y_test = train_test_split(Y, test_size=0.2, random_state=21)

In [ ]:
def get_results(n_components,max_features,criterion,n_estimators,X,Y):
    kf = KFold(n=len(X), n_folds=12, shuffle=True, random_state=21)
    d = []
    for train_index, test_index in kf:
        for rf_features,criter,rf_estimators in itertools.product(max_features,criterion,n_estimators):
            X_tr, X_test = X[train_index], X[test_index]
            X_train, X_validation = train_test_split(X_tr, test_size=0.2, random_state=21)
            Y_tr, Y_test = Y[train_index], Y[test_index]
            Y_train, Y_validation = train_test_split(Y_tr, test_size=0.2, random_state=21)
            del X_tr
            del Y_tr
            forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion=criter,
                    max_depth=None, max_features=rf_features, max_leaf_nodes=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=rf_estimators, n_jobs=1,
                    oob_score=False, random_state=21, verbose=0,
                    warm_start=False)
            forest.fit(X_train,Y_train)
            forest_pred = forest.predict(X_validation)
            name = '%s,%s,%s'%(str(rf_features),str(criter),str(rf_estimators))
            d.append((name,roc_auc_score(Y_validation,forest_pred),X_train,X_test,Y_train,Y_test))
    return sorted(d, key=lambda tup: tup[1])[-1]

In [ ]:
n_components = [1,2,3]
max_features = ['auto','sqrt','log2']
criterion = ['gini','entropy']
n_estimators = [3,5,9,12,20,35] # Check some possible estimators
n_estimators = [35,40,45]

# Get the best parameters
best = get_results(n_components,max_features,criterion,n_estimators,X.as_matrix(),Y.as_matrix())
best_params,X_train_best,X_test_best,Y_train_best,Y_test_best = best[0].split(','),best[2],best[3],best[4],best[5]

# Run RF
forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion=best_params[1],
                max_depth=None, max_features=best_params[0], max_leaf_nodes=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=int(best_params[2]), n_jobs=1,
                oob_score=False, random_state=21, verbose=0,
                warm_start=False)
forest.fit(X_train_best,Y_train_best)
Data_filt['LabelsRF'] = forest.predict(X)

# Get results
print(accuracy_score(Y_test_best,forest.predict(X_test_best)))
print(roc_auc_score(Y_test_best,forest.predict(X_test_best)))

In [ ]:
print(confusion_matrix(Y, Data_filt['LabelsRF']))

In [ ]:
# http://blog.datadive.net/selecting-good-features-part-iii-random-forests/

In [ ]:
Data_filt.head()

In [ ]:
Data_filt.LabelsRF.value_counts()

In [ ]:
from PIL import Image
file_name='Raster_images/Land.png'
m=numpy.array(Image.open(file_name)).shape[0]
n=numpy.array(Image.open(file_name)).shape[1]
def indexes(m,n):    
    index_2dim={}
    for M in range(0,m*n):
        index_2dim[M]={'i':int(M)/int(n), 'j':M-((int(M)/int(n))*n)}
    return index_2dim
original_index_dict=indexes(m,n)

In [ ]:
#0 disease, 1 no disease, 2 outside singapore 
matrix = np.zeros((m,n))+2
for M in Data_filt.index:
    i=original_index_dict[M]['i']
    j=original_index_dict[M]['j']
    matrix[i][j]=Data_filt.LabelsRF[M]
m_l=matrix.tolist()
fig, ax = subplots(figsize=(12, 12))
plt.imshow(m_l, interpolation='nearest', cmap='hot', extent=(0.5,10.5,0.5,10.5))
plt.colorbar()
#plt.show()

In [ ]:
# Get the best parameters
best_ = get_results(n_components,max_features,criterion,n_estimators,X[['BusStop','Street','Trash']].as_matrix(),Y.as_matrix())
best_params,X_train_best,X_test_best,Y_train_best,Y_test_best = best_[0].split(','),best_[2],best_[3],best_[4],best_[5]

# Run RF
forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion=best_params[1],
                max_depth=None, max_features=best_params[0], max_leaf_nodes=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=int(best_params[2]), n_jobs=1,
                oob_score=False, random_state=21, verbose=0,
                warm_start=False)
forest.fit(X_train_best,Y_train_best)
#Data_filt['LabelsRF'] = forest.predict(X)

# Get results
print accuracy_score(Y_test_best,forest.predict(X_test_best))
print roc_auc_score(Y_test_best,forest.predict(X_test_best))